## Liberías a utilizar

In [5]:
import numpy as np
from scipy.integrate import odeint
import random
import pandas as pd

## Funciones Modelo y Verosimilitud

In [6]:
"""
Funciones para el modelo SIR
"""
# MODELO
def modelo(y, t, beta, gamma, eta, mu, theta, N):
    S, E, I, R = y
    dSdt = mu * N - (beta * S * I / N) - mu * S + theta * R
    dEdt = (beta * S * I / N) - (mu + gamma) * E
    dIdt = gamma * E - (mu + eta) * I
    dRdt = eta * I - (mu + theta) * R
    return [dSdt, dEdt, dIdt, dRdt]

# Modelo en el tiempo ****PRUEBA****
def sim_mod_time(beta, gamma, eta, mu, theta, S0, E0, I0, R0, N, time):
    # Cond Iniciales
    y0 = [S0, E0, I0, R0]
    # Resolvemos el sistema
    ret = odeint(modelo, y0, time, args=(beta, gamma, eta, mu, theta, N))
    # Obtenemos los valores de I para luego compararlos con los datos reales
    S, E, I, R = ret.T
    return I

"""
Funciones estadísticas
"""

# Verosimilitud que nos ayude a comparar los datos reales con los simulados
def likelihood(data, predicted_cases):
    # Asumimos normalidad de errores xd
    sigma = np.std(data) 
    return -0.5 * np.sum((data - predicted_cases)**2 / sigma**2)


## MCMC

Utilizaremos Metroplosi Hasting, un método MCMC que propone nuevos valores de estimadores y decide si los acepta o rechaza.

In [7]:
"""
MCMC
"""

# !!******PRUEBA 1 DE MCMC******!!
def mcmc(data, param_0, iter, time):
    # Parametros iniciales la neta completamente aleatorios jajsjdlkajdfkjasdf
    beta, gamma, eta, mu, theta = param_0
    # Parametros aceptados
    accepted_params = []
    # Verosimilitud actual
    current_likelihood = likelihood(data, sim_mod_time(beta, gamma, eta, mu, theta, S0, E0, I0, R0, N, time))
    
    for i in range(iter):
        # Se proponen nuevos parametros, con una distribucion uniforme, moviendolos poquito namas
        new_params = [
            beta + random.uniform(-0.01, 0.01),
            gamma + random.uniform(-0.01, 0.01),
            eta + random.uniform(-0.01, 0.01),
            mu + random.uniform(-0.001, 0.001),
            theta + random.uniform(-0.01, 0.01)
        ]
        
         
        new_predicted = sim_mod_time(*new_params, S0, E0, I0, R0, N, time)
        
        # Calculate new likelihood
        new_likelihood = likelihood(data, new_predicted)
        
        # Acceptance probability
        accept_prob = np.exp(new_likelihood - current_likelihood)
        
        if accept_prob > random.uniform(0, 1):
            # Accept new parameters
            beta, gamma, eta, mu, theta = new_params
            current_likelihood = new_likelihood
            accepted_params.append(new_params)
    
    return accepted_params

In [9]:
cleaned_data = pd.read_csv('cleaned_epidemiology_data.csv')

param_0 = [0.2, 0.1, 0.05, 0.001, 0.01]

time = np.linspace(0, len(cleaned_data), len(cleaned_data)) 

S0 = 999900
E0 = 100
I0 = cleaned_data['new_confirmed'].iloc[0]
R0 = 0
N = S0 + E0 + I0 + R0 


iter = 5000
mcmc_results = mcmc(cleaned_data['new_confirmed'].values, param_0, iter , time)


mcmc_results[:5]


C:\Users\lebue\AppData\Local\Temp\ipykernel_4308\3102774617.py:31: RuntimeWarning: overflow encountered in exp
  accept_prob = np.exp(new_likelihood - current_likelihood)
c:\Users\lebue\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\integrate\_odepack_py.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


[[0.19373077024608593,
  0.09187719853902672,
  0.059405129303032926,
  0.0013144144048149217,
  0.0020634383964109153],
 [0.20242729763371128,
  0.08742397598050501,
  0.06833770005584296,
  0.0019368021456473744,
  -0.003791734181811734],
 [0.20256314910711182,
  0.08126061286019164,
  0.07678043648263945,
  0.0009989040704251827,
  -0.0011154754125369901],
 [0.1977049614565995,
  0.0807500242548351,
  0.07645773616029292,
  0.001375317509198191,
  -0.002502627617136252],
 [0.19855827101580092,
  0.08376186630558244,
  0.08567778426625125,
  0.0018984246423900423,
  -0.010854366753285717]]